# Memory management

Numerical software tends to use as much memory as a workstation has.  The memory has two major uses: (i) to hold the required huge amount of data, and (ii) to gain speed.

1. Linux memory model: stack, heap, and memory map
2. C memory management API
3. C++ memory management API
4. STL allocator API
3. Object counter

Modern computers use a hierarchical memory system.  Registers locate in the processor chip and are the fastest and scarest memory.  There is no additional cycle needed for the CPU to access the bits in registers.

Farther from the CPU, we have cache memory in multiple levels.  It takes 1 to 30 cycles to get data from cache memory to CPU, depending on the level.  Then we reach the main memory.  Data in main memory takes 50-200 cycles of latency before getting to CPU.

# Register, stack, heap, and memory map

All data in a computer program takes space in memory.  Depending on the usage, we will allocate them in different places.  The fundamental data types, the numbers to be crunched, will eventually go into the register file.  Temporary small objects are allocated on the stack.  Data to be shared among functions go to dynamically allocated memory.  Depending on the size, the memory manager may choose to use heap or memory map (mmap).

When talking about memory management, we usually mean dynamic memory management.  Large chunks of static memory in an executable image shouldn't be used.

# C dynamic memory

The C programming language defines 5 API for manipulate dynamic memory.

* `void * malloc(size_t size);`
* `void * calloc(size_t num, size_t size);`
* `void * realloc(void * ptr, size_t new_size);`
* `void free(void* ptr);`
* `void * aligned_alloc(size_t alignment, size_t size);`

For conveniece, we call a library or part of a library that implements the dynamic memory management APIs a memory manager.  Although we should focus on C++ code, it is crucial to know how a C memory manager works, because

1. C++ memory managers are implemented by C.
2. C memory management API can do what C++ cannot.
3. C memory managers sometimes are faster than C++.

```cpp
int64_t * inner();

void outer()
{
    printf("frame address of outer: %p\n", __builtin_frame_address(0));

    int64_t * data = inner();
    printf("data returned from inner: %p\n", &data);

    for (size_t it = 0; it < 32; ++it)
    {
        if (data[it] != 200 + it)
        {
            printf("error\n");
        }
    }
    printf("=== malloc tested\n");

    // You must free the memory after you finish using it.  Otherwise it will
    // remain in the process unclaimed, results in the "memory leak".
    free(data);
    printf("=== free tested\n");

    /* You may not use the memory that is already freed.  The results is
     * undefined.
    for (size_t it = 0; it < 32; ++it)
    {
        if (data[it] != 200 + it)
        {
            printf("error\n");
        }
    }
    */

    // The following two allocations result in the same zero-initialized array.
    // But when the OS has already zero initialized the memory, calloc may run
    // faster than malloc-then-fill.
    data = (int64_t *) calloc(32, sizeof(int64_t));
    free(data);
    data = (int64_t *) malloc(32 * sizeof(int64_t));
    for (size_t it = 0; it < 32; ++it) { data[it] = 0; }
    free(data);
    printf("=== calloc tested\n");

    // Reallocate the memory with smaller or larger size.
    data = (int64_t *) malloc((1UL << 20) * 2 * sizeof(int64_t));
    printf("address by malloc: %p\n", data);
    data = (int64_t *) realloc(data, (1UL << 20) * 1 * sizeof(int64_t));
    printf("address by realloc to smaller memory: %p\n", data);
    data = (int64_t *) realloc(data, (1UL << 20) * 4 * sizeof(int64_t));
    printf("address by realloc to larger memory: %p\n", data);
    free(data);
    printf("=== realloc tested\n");

    // Aligned allocation.
    int64_t * data1 = (int64_t *) malloc(sizeof(int64_t));
    printf("address by malloc: %p\n", data1);
    int64_t * data2 = (int64_t *) aligned_alloc(256, 256 * sizeof(int64_t));
    printf("address by aligned_alloc: %p\n", data2);
    free(data1);
    free(data2);
    printf("=== aligned_alloc tested\n");
}

int64_t * inner()
{
    printf("frame address of inner: %p\n", __builtin_frame_address(0));

    // An array on the stack.  It is popped away when execution leaves this
    // function.  You cannot use the memory outside this function.
    int64_t data_stack[32];

    for (size_t it = 0; it < 32; ++it)
    {
        data_stack[it] = 100 + it;
    }
    printf("stack memory: %p\n", data_stack);

    // A dynamic array.
    int64_t * data_dynamic = (int64_t *) malloc(32 * sizeof(int64_t));

    for (size_t it = 0; it < 32; ++it)
    {
        data_dynamic[it] = 200 + it;
    }
    printf("dynamic memory: %p\n", data_dynamic);

    return data_dynamic;
}

int main(int argc, char ** argv)
{
    printf("frame address of main: %p\n", __builtin_frame_address(0));

    outer();

    return 0;
}
```

In [1]:
!make cmem; echo "--- built; run:"; ./cmem

gcc -std=c11 -O3 -g -o cmem.o -c cmem.c
gcc -std=c11 -O3 -g -o cmem cmem.o
--- built; run:
frame address of main: 0x7ffeec5c0fc0
frame address of outer: 0x7ffeec5c0fb0
frame address of inner: 0x7ffeec5c0f70
stack memory: 0x7ffeec5c0e60
dynamic memory: 0x7fd7c1c02b90
data returned from inner: 0x7ffeec5c0f90
=== malloc tested
=== free tested
=== calloc tested
address by malloc: 0x103657000
address by realloc to smaller memory: 0x103657000
address by realloc to larger memory: 0x104658000
=== realloc tested
address by malloc: 0x7fd7c1c02b90
address by aligned_alloc: 0x7fd7c2002000
=== aligned_alloc tested


# C++ dynamic memory: new and delete

Objects in C++ have 4 storage durations:

1. static
2. thread
3. automatic
4. dynamic

The first 3 of them, static, thread, and automatic storage durations, are distinguished by the declarations.  The last one, dynamic storage duration, is managed by `operator new/delete` and our focus in memory management.

There are 3 frequent use cases of the `new/delete` expression:

1. Single object allocation.
2. Array allocation.
3. Placement new.

Precisely speaking, only the first two cases are fully related to memory management.  The third use case doesn't directly allocate or deallocate memory, but allows to use the `new/delete` expression for constructing objects on an already-allocated block of memory.

```cpp
void scalar_form()
{
    std::cout
        << "frame address of scalar_form: " << __builtin_frame_address(0)
        << std::endl;

    // Doing this place 8k bytes on stack.
    Block block_stack;
    for (size_t it = 0; it < 1024; ++it)
    {
        block_stack.buffer[it] = 1000 + it;
    }
    std::cout << "object on stack: " << &block_stack << std::endl;
    std::cout
        << "address difference: "
        << reinterpret_cast<std::size_t>(__builtin_frame_address(0))
         - reinterpret_cast<std::size_t>(&block_stack)
        << ", sizeof(Block): " << sizeof(Block)
        << std::endl;

    // Use the new expression.  Note that this "new" is an expression.  It
    // calls the operator ("::operator new"), but not the operator itself.
    Block * block_dynamic = new Block;
    std::cout << "object on dynamic memory: " << block_dynamic << std::endl;

    for (size_t it = 0; it < 1024; ++it)
    {
        block_dynamic->buffer[it] = 2000 + it;
    }
    std::cout << "=== new tested" << std::endl;

    // The delete expression that destruct and deallocate the memory of the
    // dynamic block object.  Similarly, the expression calls ::operator delete
    // for block_dynamic.
    delete block_dynamic;
    std::cout << "=== delete tested" << std::endl;
}
```

```cpp
void array_form()
{
    // An array on the stack.  It is popped away when execution leaves this
    // function.  You cannot use the memory outside this function.
    int64_t data_stack[32];

    for (size_t it = 0; it < 32; ++it)
    {
        data_stack[it] = 100 + it;
    }
    std::cout << "stack array memory: " << data_stack << std::endl;

    // A dynamic array.
    int64_t * data_dynamic = new int64_t[32];

    for (size_t it = 0; it < 32; ++it)
    {
        data_dynamic[it] = 200 + it;
    }
    std::cout << "dynamic array memory: " << data_dynamic << std::endl;
    std::cout << "=== new[] tested" << std::endl;

    delete[] data_dynamic;
    std::cout << "=== delete[] tested" << std::endl;
}
```

```cpp
void placement()
{
    char * buffer = new char[sizeof(Block)];

    Block * block = new (buffer) Block;
    for (size_t it = 0; it < 1024; ++it)
    {
        block->buffer[it] = it;
    }
    std::cout << "=== placement new tested" << std::endl;

    /*
     * This induces undefined behavior.  Don't do this.
    delete block;
    */

    // Instead of deleting the pointer block, call explicit the destructor and
    // delete the original buffer.
    block->~Block();
    delete[] buffer;
}
```

In [2]:
!make cppmem; echo "--- built; run:"; ./cppmem

g++ -std=c++17 -O3 -g -o cppmem.o -c cppmem.cpp
g++ -std=c++17 -O3 -g -o cppmem cppmem.o
--- built; run:
frame address of scalar_form: 0x7ffeec46dfb0
Block (0x7ffeec46bf90) constructed
object on stack: 0x7ffeec46bf90
address difference: 8224, sizeof(Block): 8192
Block (0x7fac75002000) constructed
object on dynamic memory: 0x7fac75002000
=== new tested
Block (0x7fac75002000) destructed
=== delete tested
Block (0x7ffeec46bf90) destructed
stack array memory: 0x7ffeec46de90
dynamic array memory: 0x7fac74c02b90
=== new[] tested
=== delete[] tested
Block (0x7fac75002000) constructed
=== placement new tested
Block (0x7fac75002000) destructed


# STL allocator

STL uses another set of template API for allocating the memory for most of its container.  By default, the STL containers use ``std::allocator`` class template for memory allocation.  We are allowed to provide custom allocators to the containers.

```cpp
struct ByteCounterImpl
{

    std::atomic_size_t allocated = 0;
    std::atomic_size_t deallocated = 0;
    std::atomic_size_t refcount = 0;

}; /* end struct ByteCounterImpl */

/**
 * One instance of this counter is shared among a set of allocators.
 *
 * The counter keeps track of the bytes allocated and deallocated, and report
 * those two numbers in addition to bytes that remain allocated.
 */
class ByteCounter
{

public:

    ByteCounter()
      : m_impl(new ByteCounterImpl)
    { incref(); }

    ByteCounter(ByteCounter const & other)
      : m_impl(other.m_impl)
    { incref(); }

    ByteCounter & operator=(ByteCounter const & other)
    {
        if (&other != this)
        {
            decref();
            m_impl = other.m_impl;
            incref();
        }

        return *this;
    }

    ByteCounter(ByteCounter && other)
      : m_impl(other.m_impl)
    { other.decref(); }

    ByteCounter & operator=(ByteCounter && other)
    {
        if (&other != this)
        {
            decref();
            m_impl = other.m_impl;
        }

        return *this;
    }

    ~ByteCounter() { decref(); }

    void swap(ByteCounter & other)
    {
        std::swap(m_impl, other.m_impl);
    }

    void increase(std::size_t amount)
    {
        m_impl->allocated += amount;
    }

    void decrease(std::size_t amount)
    {
        m_impl->deallocated += amount;
    }

    std::size_t bytes() const { return m_impl->allocated - m_impl->deallocated; }
    std::size_t allocated() const { return m_impl->allocated; }
    std::size_t deallocated() const { return m_impl->deallocated; }
    /* This is for debugging. */
    std::size_t refcount() const { return m_impl->refcount; }

private:

    void incref() { ++m_impl->refcount; }

    void decref()
    {
        if (nullptr == m_impl)
        {
            // Do nothing.
        }
        else if (1 == m_impl->refcount)
        {
            delete m_impl;
            m_impl = nullptr;
        }
        else
        {
            --m_impl->refcount;
        }
    }

    ByteCounterImpl * m_impl;

}; /* end class ByteCounter */

/**
 * Very simple allocator that counts the number of bytes allocated through it.
 */
template <class T>
struct MyAllocator
{

    using value_type = T;

    // Just use the default constructor of ByteCounter for the data member
    // "counter".
    MyAllocator() = default;

    template <class U> constexpr
    MyAllocator(const MyAllocator<U> & other) noexcept
    {
        counter = other.counter;
    }

    T * allocate(std::size_t n)
    {
        if (n > std::numeric_limits<std::size_t>::max() / sizeof(T))
        {
            throw std::bad_alloc();
        }
        const std::size_t bytes = n*sizeof(T);
        T * p = static_cast<T *>(std::malloc(bytes));
        if (p)
        {
            counter.increase(bytes);
            return p;
        }
        else
        {
            throw std::bad_alloc();
        }
    }

    void deallocate(T* p, std::size_t n) noexcept
    {
        std::free(p);

        const std::size_t bytes = n*sizeof(T);
        counter.decrease(bytes);
    }

    ByteCounter counter;

}; /* end struct MyAllocator */

template <class T, class U>
bool operator==(const MyAllocator<T> & a, const MyAllocator<U> & b)
{
    return a.counter == b.counter;
}

template <class T, class U>
bool operator!=(const MyAllocator<T> & a, const MyAllocator<U> & b)
{
    return !(a == b);
}
```

In [3]:
!make alloc; echo "--- built; run:"; ./alloc

g++ -std=c++17 -O3 -g -o alloc.o -c alloc.cpp
g++ -std=c++17 -O3 -g -o alloc alloc.o
--- built; run:
allocator: bytes = 0 allocated = 0 deallocated = 0
allocator: bytes = 8192 allocated = 16376 deallocated = 8184
allocator: bytes = 0 allocated = 16376 deallocated = 16376


# Instance counter

In some cases, we want to know how many intances are created of certain classes.  One quick way is to add an instance counter for the specific class.  We may immediately know the number at any given time point of the execution.

```cpp
template <class T>
class InstanceCounter
{

public:

    InstanceCounter() { ++m_constructed; }
    InstanceCounter(InstanceCounter const & other) { ++m_constructed; }
    ~InstanceCounter() { ++m_destructed; }

    static std::size_t active() { return m_constructed - m_destructed; }
    static std::size_t constructed() { return m_constructed; }
    static std::size_t destructed() { return m_destructed; }

private:

    static std::atomic_size_t m_constructed;
    static std::atomic_size_t m_destructed;

}; /* end class InstanceCounter */

template <class T> std::atomic_size_t InstanceCounter<T>::m_constructed = 0;
template <class T> std::atomic_size_t InstanceCounter<T>::m_destructed = 0;
```

In [4]:
!make icount; echo "--- built; run:"; ./icount

g++ -std=c++17 -O3 -g -o icount.o -c icount.cpp
g++ -std=c++17 -O3 -g -o icount icount.o
--- built; run:
instance: active = 1 constructed = 1 destructed = 0
instance: active = 65 constructed = 65 destructed = 0
instance: active = 1 constructed = 65 destructed = 64
instance: active = 0 constructed = 65 destructed = 65


# Exercises

1. Calling `delete` on the address returned by `new[]` may cause problems.  Write a program and analyze what the problems may be.
2. When using a single thread, what is the runtime overhead of the instance counting technique?  Write a program and analyze.